# Filter columns and rows

- [Display feature summaries](#Display-feature-summaries)
- [Explore features](#Explore-features)
- [Filter data](#Filter-data)

The most recent data in the LendingClub
[dataset](https://www.kaggle.com/datasets/wordsforthewise/lending-club) is from 2018,
and since then, LendingClub has [stopped operating as a peer-to-peer
lender](https://en.wikipedia.org/wiki/LendingClub#End_of_P2P_platform,_2019-2020).
Unsurprisingly, it's difficult to find explanations on the LendingClub website about the
features in this dataset.

Sites not officially associated with LendingClub still contain information about the
peer-to-peer service previously offered by LendingClub.  As a result, the feature
exploration for this project includes links to miscellaneous pages such as blogs.

Beginning with the current notebook, however, lack of detailed information about
features does impose some limits.  For instance, rows containing certain values of
`loan_status` are filtered out simply because it is difficult to understand what those
values mean.

This notebook does the following:

- Explore features to determine what filtering should be done.
- Filter out certain columns and rows from the data on accepted loans.

The information available on rejected loans is fairly limited, so for now, I will limit
attention to the data on accepted loans.

The data-cleaning steps developed in this notebook have been incorporated into the
project's Python package `notebook_tools`.

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
from IPython.display import display

from notebook_tools.data_cleaning import (
    convert_acc_loan_data,
    convert_rej_loan_data,
    load_acc_loan_data,
    load_acc_loan_feat_desc,
    load_rej_loan_data,
)
from notebook_tools.feature_exploration import (
    get_group_sizes,
    get_value_counts,
    style_loan_summary,
    style_value_counts,
    summarize_acc_loans,
    summarize_loan_data,
)

## Display feature summaries

Use functions in the package `notebook_tools` to load data and generate feature
summaries.

In [ ]:
acc_loan_data = load_acc_loan_data(excluded_cols=["member_id"]).pipe(
    convert_acc_loan_data
)

In [ ]:
acc_loan_feat_desc = load_acc_loan_feat_desc()

In [ ]:
rej_loan_data = load_rej_loan_data().pipe(convert_rej_loan_data)

### Feature summaries for accepted loans

#### Total number of records:  2,260,701

In [ ]:
print(f"The number of records for accepted loans is {len(acc_loan_data.index):,d}.")

In [ ]:
for dtype in [np.number, "string", "boolean"]:
    summary = summarize_acc_loans(acc_loan_data, dtype, acc_loan_feat_desc)
    print(f"\n\nThe number of features of type {dtype} is {len(summary.index)}.\n\n")
    display(style_loan_summary(summary))

### Feature summaries for rejected loans

#### Total number of records:  27,648,741

In [ ]:
print(f"The number of records for accepted loans is {len(rej_loan_data.index):,d}.")

In [ ]:
for dtype in [np.number, "string"]:
    summary = summarize_loan_data(rej_loan_data, dtype)
    display(style_loan_summary(summary))

## Explore features

### `policy_code` / `Policy Code`

What do the columns `policy_code` (for accepted loans) and `Policy Code` (for rejected
loans) refer to?

From ["What are these Policy Code 2 Loans at Lending
Club?"](https://www.fintechnexus.com/policy-code-2-loans-lending-club/):

> - These [Policy Code 2 loans] are loans made to borrowers that do not meet Lending
Club’s current credit policy standards.
> - The FICO scores on these borrowers are typically 640-659, below the 660 threshold on
Policy Code 1 loans.
> - These loans are made available to select institutional investors who have a great
deal of experience with consumer loans in this credit spectrum and with Lending Club.

In [ ]:
policy_code_counts = get_value_counts(acc_loan_data["policy_code"])
display(style_value_counts(policy_code_counts))

In [ ]:
policy_code_counts_rej = get_value_counts(rej_loan_data["Policy Code"])
display(style_value_counts(policy_code_counts_rej))

### `loan_status`

What are the distinct values for the column `loan_status`?

In [ ]:
loan_status_counts = get_value_counts(acc_loan_data["loan_status"])
display(style_value_counts(loan_status_counts))

The 33 rows that have `NA` for `loan_status` also have `NA` for all other features other
than `id`, so these rows can be filtered from the data.

From the values of `id` displayed in the output of next cell, these null rows appear to
be associated with the policy code.

In [ ]:
missing_status = acc_loan_data[acc_loan_data["loan_status"].isna()]

In [ ]:
display(missing_status.head(4).transpose())

As a check, verify that if the `id` column is dropped, then all values are `NA` in rows
that are missing `loan_status`.

In [ ]:
display(missing_status.drop("id", axis="columns").count().sum())

Create a dataframe that has these empty rows filtered out.  After additional filtering,
this dataframe will be used to recreate the SQLite database.

In [ ]:
filtered_loan_data = acc_loan_data[acc_loan_data["loan_status"].notna()]

In [ ]:
loan_status_counts = get_value_counts(filtered_loan_data["loan_status"])
display(style_value_counts(loan_status_counts))

Note that after the rows with missing `loan_status` have been filtered out, there are no
missing values for `policy_code`.  Since all rows have the same value for `policy_code`,
this column can be dropped.

In [ ]:
policy_code_counts = get_value_counts(filtered_loan_data["policy_code"])
display(style_value_counts(policy_code_counts))

Most of the values for `loan_status` are explained at ["What Do the Different Note Statuses
Mean?"](https://www.lendingclub.com/help/investing-faq/what-do-the-different-note-statuses-mean).

However, the values `Does not meet the credit policy. Status:Fully Paid` and `Does not
meet the credit policy. Status:Charged Off` are unclear. Let's take look at a random
sample of the rows that have these value of loan status.

In [ ]:
bool_index = filtered_loan_data["loan_status"].str.endswith("Status:Fully Paid")
sampled_data = filtered_loan_data[bool_index].sample(
    n=5, random_state=59147, axis="index"
)
with pd.option_context("display.max_columns", None):
    display(sampled_data)

In [ ]:
bool_index = filtered_loan_data["loan_status"].str.endswith("Status:Charged Off")
sampled_data = filtered_loan_data[bool_index].sample(
    n=5, random_state=59147, axis="index"
)
with pd.option_context("display.max_columns", None):
    display(sampled_data)

Nothing jumps out from this small random sample.  Rather than trying to guess why
certain rows do not meet the credit policy, I'll exclude these rows.

In [ ]:
bool_index = filtered_loan_data["loan_status"].str.startswith("Does not meet")
filtered_loan_data = filtered_loan_data[~bool_index]

In [ ]:
loan_status_counts = get_value_counts(filtered_loan_data["loan_status"])
display(style_value_counts(loan_status_counts))

### `issue_d`

The description of this feature is "The month which the loan was funded".

After rows with problematic values of `loan_status` have been filtered out, there are no
missing values for `issue_d`.

In [ ]:
filtered_loan_data["issue_d"].isna().sum()

In [ ]:
to_plot = get_group_sizes(filtered_loan_data, group_by="issue_d")
fig = px.line(
    to_plot,
    x="issue_d",
    y="count",
    markers=True,
    labels={"issue_d": "Loan date", "count": "Number of loans"},
    hover_data={"count": ":.3s"},
    title="Number of accepted loans by date",
)
fig.show()

I will exclude pre-2012 dates from the analysis.

Analysis and prediction based on this data will need to take account of changes in
behavior over time, and given the relatively small number of loans issued before 2012,
it is not worthwhile to include the pre-2012 data.

In [ ]:
bool_index = filtered_loan_data["issue_d"] >= "2012-01"
filtered_loan_data = filtered_loan_data[bool_index]

In [ ]:
to_plot = get_group_sizes(filtered_loan_data, group_by="issue_d")
fig = px.line(
    to_plot,
    x="issue_d",
    y="count",
    markers=True,
    labels={"issue_d": "Loan date", "count": "Number of loans"},
    hover_data={"count": ":.3s"},
    title="Number of accepted loans by date",
)
fig.show()

### `loan_amnt` / `funded_amnt` / `funded_amnt_inv`

What is the distinction between `loan_amnt`, `funded_amnt`, `funded_amnt_inv`?

Start by examining the feature descriptions.

In [ ]:
amount_features = acc_loan_feat_desc.loc[
    ["loan_amnt", "funded_amnt", "funded_amnt_inv"], ["description"]
]
display(style_loan_summary(amount_features))

What should we infer in cases where `loan_amnt` is different than `funded_amnt`, or in
cases where `funded_amnt` is different than `funded_amnt_inv`?  It's not completely
clear from these descriptions.

Investigate the frequency of these cases.

In [ ]:
# First check for missing values.
for column_name in ["loan_amnt", "funded_amnt", "funded_amnt_inv"]:
    na_count = filtered_loan_data[column_name].isna().sum()
    print(f'\nThe number of missing values for feature "{column_name}" is {na_count}.')

In [ ]:
bool_index = (filtered_loan_data["loan_amnt"] - filtered_loan_data["funded_amnt"]) != 0
print(
    '\nThe number of loans with "loan_amnt" different than "funded_amnt" is '
    f"{sum(bool_index)}.\n"
)

to_plot = get_group_sizes(filtered_loan_data[bool_index], group_by="issue_d")
fig = px.scatter(
    to_plot,
    x="issue_d",
    y="count",
    labels={"issue_d": "Loan date", "count": "Number of loans"},
    hover_data={"count": ":,d"},
    title='Number of loans with "loan_amnt" different than "funded_amnt"',
)
fig.show()

In [ ]:
bool_index = (
    filtered_loan_data["funded_amnt"] - filtered_loan_data["funded_amnt_inv"]
) != 0
print(
    '\nThe number of loans with "funded_amnt" different than "funded_amnt_inv" is '
    f"{sum(bool_index)}.\n"
)

to_plot = get_group_sizes(filtered_loan_data[bool_index], group_by="issue_d")
fig = px.scatter(
    to_plot,
    x="issue_d",
    y="count",
    labels={"issue_d": "Loan date", "count": "Number of loans"},
    hover_data={"count": ":,d"},
    title='Number of loans with "funded_amnt" different than "funded_amnt_inv"',
)
fig.show()

Discussion:

- Only 68 of the 2.2 million loans have `loan_amnt` different than `funded_amnt`.
Essentially all the loans are fully funded.
- About 130k of the loans have different values for `funded_amnt` and `funded_amnt_inv`.
Is LendingClub itself providing funding in these case?

While I don't understand the cause of the differences between `loan_amnt`,
`funded_amnt`, and `funded_amnt_inv`, I won't filter out the rows with different values
for these features.  Unlike the rows where `loan_status` includes the string `"Does not
meet the credit policy"`, there isn't a strong indication that rows with different
values for `loan_amnt`, `funded_amnt`, and `funded_amnt_inv` are fundamentally
problematic.

### `initial_list_status`

The feature `initial_list_status` is explained in [this blog
post](https://sirallen.name/blog/note-on-lending-club/):

> The variable initial_list_status is available in the public data and identifies
> whether a loan was initially listed in the whole (W) or fractional (F) market. Loans
> listed “whole” become available for fractional funding (and vice versa) if there are no
> buyers within a certain time frame.

In [ ]:
list_status_counts = get_value_counts(filtered_loan_data["initial_list_status"])
display(style_value_counts(list_status_counts))

Given this explanation of the feature `initial_list_status`, there's no need to drop the
feature or filter out rows based on the value of the feature

### `int_rate`

The notebook `analysis-01.ipynb` presents an in-depth analysis of interest rates for the
accepted loans. While doing that analysis, I found that some loans had an anomalously
low interest rate, given the poor grade assigned to the loans by LendingClub.

Here I present analysis showing that some interest rates are anomalously low, and I
explore possible explanations. Since none of the explanations are well supported by the
data, I filter out the corresponding loans.

Note on the code for visualization:  I use plotly histograms to highlight the anomalous
interest rates, and the binning for these histograms needs to be done outside of plotly.
The reason is that plotly does binning in JavaScript, and so unbinned data passed to
plotly's histogram function becomes part of the javascript code stored with the
notebook. For the current data set, this can increase the notebook size on disk by a
factor of more than 100.

In [ ]:
min = filtered_loan_data["int_rate"].min()
max = filtered_loan_data["int_rate"].max()
print(
    'The minimum and maximum values of "int_rate" '
    f"are {min} and {max}, respectively."
)

In [ ]:
# Define arrays / lists needed for binning the histogram and plotting the bins in
# plotly.
int_rate_bins = np.linspace(5, 31, num=27)
int_rate_bin_labels = [f"{left:d}% - {left+0.99:.2f}%" for left in range(5, 31)]
int_rate_tick_vals = int_rate_bin_labels[0::5]
int_rate_tick_text = [f"{left}%" for left in range(5, 35, 5)]

In [ ]:
filtered_loan_data["int_rate_bin"] = pd.cut(
    filtered_loan_data["int_rate"],
    bins=int_rate_bins,
    labels=int_rate_bin_labels,
    right=False,
)

In [ ]:
to_plot = get_group_sizes(filtered_loan_data, group_by="int_rate_bin")
fig = px.bar(
    to_plot,
    x="int_rate_bin",
    y="count",
    labels={"count": "Number of loans", "int_rate_bin": "Interest rate"},
    title="Distribution of loan interest rate",
)
customdata = to_plot["int_rate_bin"]
hovertemplate = "Interest rate=%{customdata}<br>Number of loans=%{y:.3s}<extra></extra>"
fig.update_traces(customdata=customdata, hovertemplate=hovertemplate)
fig.update_layout(bargap=0)
fig.update_xaxes(
    tickmode="array", tickvals=int_rate_tick_vals, ticktext=int_rate_tick_text
)
fig.show()

In [ ]:
to_plot = get_group_sizes(filtered_loan_data, group_by=["int_rate_bin", "grade"])
fig = px.bar(
    to_plot,
    x="int_rate_bin",
    y="count",
    facet_row="grade",
    labels={
        "count": "Number of loans",
        "int_rate_bin": "Interest rate",
        "grade": "Grade",
    },
    title="Distribution of loan interest rate by loan grade",
    hover_data={"count": ":.3s"},
    height=1200,
)
fig.update_xaxes(tickmode="array", tickvals=int_rate_tick_vals, tickangle=45)
fig.update_yaxes(matches=None, title="")
fig.update_layout(bargap=0, yaxis4_title="Number of Loans")
fig.show()

In [ ]:
to_plot = get_group_sizes(
    filtered_loan_data, group_by=["int_rate_bin", "grade", "sub_grade"]
)
to_plot["sub_grade"] = to_plot["sub_grade"].str[1]
fig = px.bar(
    to_plot,
    x="int_rate_bin",
    y="count",
    facet_row="grade",
    color="sub_grade",
    labels={
        "count": "Number of loans",
        "int_rate_bin": "Interest rate",
        "grade": "Grade",
        "sub_grade": "Sub-grade",
    },
    title="Distribution of loan interest rate by loan grade and sub-grade",
    hover_data={"count": ":.3s"},
    height=1200,
)
fig.update_xaxes(tickmode="array", tickvals=int_rate_tick_vals, tickangle=45)
fig.update_yaxes(matches=None, title="")
fig.update_layout(bargap=0, yaxis4_title="Number of Loans")
fig.show()

The previous plots show the following:

- The interest rate varies systematically with the loan grade.
- For loans with a high grade, the interest rate varies systemtically with loan
sub-grade.
- For loans with a low grade, the dependence of interest rate on sub-grade is complex.

To get a different view of these patterns, I'll set the y-axis to show percentage of
loans rather than number of loans in each category.

In [ ]:
to_plot = get_group_sizes(
    filtered_loan_data, group_by=["int_rate_bin", "grade", "sub_grade"]
)
to_plot["sub_grade"] = to_plot["sub_grade"].str[1]
fig = px.histogram(
    to_plot,
    x="int_rate_bin",
    y="count",
    facet_row="grade",
    color="sub_grade",
    labels={
        "count": "Number of loans",
        "int_rate_bin": "Interest rate",
        "grade": "Grade",
        "sub_grade": "Sub-grade",
    },
    barnorm="fraction",
    title="Distribution of loan interest rate by loan grade and sub-grade",
    height=1200,
)


def clean_up_hovertemplate(trace):
    trace.hovertemplate = trace.hovertemplate.replace(
        "sum of Number of loans (normalized as fraction)", "Percentage"
    )


fig.for_each_trace(clean_up_hovertemplate)
fig.update_xaxes(tickmode="array", tickvals=int_rate_tick_vals, tickangle=45)
fig.update_yaxes(title="", tickformat=".2p")
fig.update_layout(bargap=0, yaxis4_title="Number of loans")
fig.show()

The plot above shows the anomalous interest rates.
For example, most of the loans in the bottom row, which correspond to the lowest grade,
have an interest rate of at least 22%, but the plot also shows a block of loans
in the bottom row with interest rate in the range of 6% - 6.99%.

Possible explanations that I explored:

- These rates are associated with a hardship plan or settlement plan.
- The funding mechanism for these loans is unusual.
- These loans are associated with some external event and were all issued at around the
same time.
- An improvement in the borrower's FICO score may have caused the interest rate to be
lowered without changing the assigned loan grade.

As shown below, I could not find support in the data for any of these guesses.

First select all of anomalous cases with interest rate in the range of 6% - 6.99%.
There are 625 of these loans.

In [ ]:
anomalous_int_rate_data = filtered_loan_data[
    (filtered_loan_data["grade"] != "A") & (filtered_loan_data["int_rate"] < 7)
]

In [ ]:
print(f"Number of loans: {len(anomalous_int_rate_data.index)}.")

Check the category of loans with grade 'D' and interest rate 8% - 8.99% shown in the
plot above.  There is only 1 loan in this category.

In [ ]:
bool_index = (filtered_loan_data["grade"] == "D") & (
    filtered_loan_data["int_rate"].between(7.99, 9, inclusive="neither")
)
print(f"Number of loans: {sum(bool_index)}.")

I will focus on the anomalous cases with interest rate in the range of 6% - 6.99%.

All of these loans have the same rate:  6.0%.

In [ ]:
anomalous_rates = list(anomalous_int_rate_data["int_rate"].unique())
print(f"Distinct values of interest rate:  {anomalous_rates}")

Only a few are associated with a hardship plan or settlement plan.

In [ ]:
display(
    style_value_counts(get_value_counts(anomalous_int_rate_data["hardship_status"]))
)
display(
    style_value_counts(get_value_counts(anomalous_int_rate_data["settlement_term"]))
)

All of these loans are fully funded.  For 46 of the loans, the value of `funded_amnt` is
different from the value of `funded_amnt_inv`, but most of the loans are fully funded by
investors.

In [ ]:
bool_index = (
    anomalous_int_rate_data["loan_amnt"] - anomalous_int_rate_data["funded_amnt"]
) != 0

print(f"The number of these loans that are not fully funded is {sum(bool_index)}.")

In [ ]:
bool_index = (
    anomalous_int_rate_data["funded_amnt"] - anomalous_int_rate_data["funded_amnt_inv"]
) != 0
print(
    'The number of these loans with "funded_amnt" different than "funded_amnt_inv" '
    f"is {sum(bool_index)}."
)

The dates of these loans are spread over the full ranges of dates of the filtered data,
so they do not appear to be associated with an external event.

In [ ]:
to_plot = get_group_sizes(anomalous_int_rate_data, group_by="issue_d")
fig = px.scatter(
    to_plot,
    x="issue_d",
    y="count",
    labels={"issue_d": "Loan date", "count": "Number of loans"},
    title="Dates of loans with anomalous interest rate 6.0%",
)
fig.show()

Look at a random sample of these records and check whether the FICO score improved from
the value it had at loan origination.

In [ ]:
# Define a reproducible random state for sampling randomly
rng = np.random.default_rng(seed=16513)

In [ ]:
sampled_data = anomalous_int_rate_data.sample(10, random_state=rng)

In [ ]:
fico_columns = [
    "fico_range_low",
    "fico_range_high",
    "last_fico_range_low",
    "last_fico_range_high",
]
style_loan_summary(sampled_data[fico_columns])

The columns `fico_range_low` and `fico_range_high` correspond to values at loan
origination, while the columns `last_fico_range_low` and `last_fico_range_high`
correspond to a more recent check of the credit rating.  This random sample does not
show a systematic jump in FICO scores for the loans with anomalously low interest rates.

Inspect the columns of the data frame for a patterns that might explain the low rates.

In [ ]:
with pd.option_context("display.max_columns", None):
    display(sampled_data)

I don't see any interesting patterns in this sample, so I will filter out the 626 rows
that have anomalously low interest rates.

In [ ]:
int_rate_is_anomalous = (filtered_loan_data["grade"] != "A") & (
    filtered_loan_data["int_rate"] < 7
)
int_rate_is_anomalous = int_rate_is_anomalous | (
    (filtered_loan_data["grade"] == "D") & (filtered_loan_data["int_rate"] < 9)
)
filtered_loan_data = filtered_loan_data[~int_rate_is_anomalous]

### `hardship_type`, `hardship_length`, and `deferral_term`

The features `hardship_type`, `hardship_length`, and `deferral_term` each have only one value (other
than &lt;NA&gt;).

In [ ]:
hardship_type_counts = get_value_counts(filtered_loan_data["hardship_type"])
display(style_value_counts(hardship_type_counts))

In [ ]:
hardship_length_counts = get_value_counts(filtered_loan_data["hardship_length"])
display(style_value_counts(hardship_length_counts))

In [ ]:
deferral_term_counts = get_value_counts(filtered_loan_data["deferral_term"])
display(style_value_counts(deferral_term_counts))

It appears that loans with &lt;NA&gt; in these columns are not associated with a hardship plan,
so each of these columns can be used as a flag for hardship plans.

If these columns are dropped, can we infer from other columns which loans are associated with a hardship plan?

Yes, because a loan has `hardship_amount` equal to &lt;NA&gt; if and only if `hardship_type`, `hardship_length`, and `deferral_term` are &lt;NA&gt;.

In [ ]:
hardship_amount_isna = filtered_loan_data["hardship_amount"].isna()
matching_na = (
    hardship_amount_isna.equals(filtered_loan_data["hardship_type"].isna())
    and hardship_amount_isna.equals(filtered_loan_data["hardship_length"].isna())
    and hardship_amount_isna.equals(filtered_loan_data["deferral_term"].isna())
)
if matching_na:
    print(
        "The columns hardship_amount, hardship_type, hardship_length, "
        "and deferral term have <NA> values at the same positions."
    )
else:
    print("Mismatch in <NA> values.")

## Filter data

Taking account of the feature summaries and the feature exploration above, certain
columns will be excluded from the analysis of accepted loans.

- url:  URL for the LC page with listing data
- title:  The loan title provided by the borrower
- desc:  Loan description provided by the borrower
- policy_code:  publicly available policy_code=1, new products not publicly available
policy_code=2
- hardship_type:  Describes the hardship plan offering
- hardship_length:  The number of months the borrower will make smaller payments than
normally obligated due to a hardship plan
- deferral_term:  Amount of months that the borrower is expected to pay less than the
contractual monthly payment amount due to a hardship plan

Also, rows will be featured out based on the following criteria:

- Problematic values for `loan_status`
    1. `<NA>`
    2. `Does not meet the credit policy. Status:Fully Paid`
    3. `Does not meet the credit policy. Status:Charged Off`
- Values of `issue_d` before 2012
- Anomalously low values of `int_rate`

In [ ]:
to_drop = [
    "url",
    "title",
    "desc",
    "policy_code",
    "hardship_type",
    "hardship_length",
    "deferral_term",
]
filtered_loan_data = filtered_loan_data.drop(
    to_drop,
    axis="columns",
)